### Сторонние библиотеки использовать нельзя

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [74]:
PAGES_FORMAT = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (512, 256),
    'A4': (297, 210),
}


class ReadableEntity:
    pass


class Journal(ReadableEntity):
    pass


class Book(ReadableEntity):
    pass


class Exporter:
    
    def export_to_txt(self, file_path):
        with open(file_path, 'w') as f:
            for key in self.__dict__:
                f.write("{}: {}".format(key, self.__dict__[key]))
     
    
class LibraryJournal(Journal, Exporter):
    pass


class LibraryBook(Book, Exporter):
    pass

### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)**

2) *вычитания* - **(0.02 балла)**

3) *умножения* - **(0.04 балла)**

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)**

5) *возведения в степень* - **(0.02 балла)** | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**

6) *представления многочлена в человеческом виде* - **(0.02 балла)**

7) *дифференцирования* - **(0.05 балла)**

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)**

**Комментарии:**

Для комплексных коэффициентов **(0.01 балла)** к каждому пункту.

Операции с числами также должны работать.

### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.